In [ ]:
import os
os.chdir(r"c:\\Users\\cledimir.semedo\\Desktop\\CodeAI")

In [ ]:
import tensorflow as tf

In [ ]:
import pandas as pd
import numpy as np
from rich import print
from scripts.scriptModel.models import build_transformer_autoencoder
autoencoder = build_transformer_autoencoder(
    seq_len=60,
    feature_dim=281,
    d_model=128,
    num_heads=4,
    dff=512,
    num_layers=2,
    dropout_rate=0.1
)

In [ ]:
autoencoder.compile(optimizer='adam', loss='mse')

In [ ]:
data = pd.read_csv("data/processed/train.csv")

In [ ]:
from sklearn.model_selection import train_test_split
train_data, val_data = train_test_split(data, test_size=0.3, random_state=42)

In [ ]:
def sequenceData(data, seq_len=60):
    """
    Transform the data into sequences of a given length.
    """
    sequences = []
    for i in range(len(data) - seq_len + 1):
        sequences.append(data[i:i + seq_len])
    return np.array(sequences,dtype=np.float32)
data_train = sequenceData(train_data[:100_000].values, seq_len=60)
data_val = sequenceData(val_data.values, seq_len=60)
print(f"Shape of sequences: {data.shape}")
print(f"Shape of training data: {data_train.shape}")

In [ ]:
autoencoder.fit(
    data_train[:10_000],
    data_train[:10_000],
    epochs=10,
    batch_size=64,
    validation_data=(data_val, data_val),
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6)
    ]
)